# BERT-CLAM 框架快速入门

本教程将指导您如何使用 BERT-CLAM 框架进行持续学习任务。

## 目录
1. 环境设置
2. 加载数据
3. 初始化模型和模块
4. 创建和组合策略
5. 注入策略并训练
6. 总结

## 1. 环境设置

首先，确保已安装所有必要的依赖：

```bash
pip install torch transformers datasets faiss-cpu
```

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from transformers import BertTokenizer
import numpy as np

# 导入BERT-CLAM组件
from bert_clam.models.bert_clam_model import BERTCLAMModel
from bert_clam.core.ewc import EnhancedElasticWeightConsolidation
from bert_clam.core.memory_replay_bank import EnhancedAdaptiveMemoryRetrieval
from bert_clam.core.grammar_aware import EnhancedGrammarAwareModule
from bert_clam.core.strategy import EWCStrategy, MRBStrategy, GrammarStrategy

print("✓ 所有依赖已成功导入")
print(f"✓ PyTorch版本: {torch.__version__}")
print(f"✓ 设备: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## 2. 加载数据

为了演示，我们创建一些虚拟数据。在实际应用中，您应该使用真实的数据集。

In [ ]:
# 初始化tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 创建虚拟数据
def create_dummy_data(num_samples=32, max_length=128):
    """创建虚拟训练数据"""
    texts = [
        "This is a positive example.",
        "This is a negative example.",
        "Another positive sentence here.",
        "Yet another negative sentence."
    ] * (num_samples // 4)
    
    labels = [1, 0, 1, 0] * (num_samples // 4)
    
    # Tokenize
    encodings = tokenizer(
        texts,
        padding='max_length',
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )
    
    # 创建数据集
    dataset = []
    for i in range(len(texts)):
        dataset.append({
            'input_ids': encodings['input_ids'][i],
            'attention_mask': encodings['attention_mask'][i],
            'labels': torch.tensor(labels[i])
        })
    
    return DataLoader(dataset, batch_size=8, shuffle=True)

# 创建两个任务的数据
task0_dataloader = create_dummy_data(num_samples=32)
task1_dataloader = create_dummy_data(num_samples=32)

print("✓ 数据加载完成")
print(f"  - 任务0: {len(task0_dataloader)} 批次")
print(f"  - 任务1: {len(task1_dataloader)} 批次")

## 3. 初始化模型和模块

创建BERT-CLAM模型实例以及各个持续学习模块。

In [ ]:
# 设置设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 初始化核心模块
ewc_module = EnhancedElasticWeightConsolidation(
    lambda_ewc=0.5,
    fisher_samples=10
)

mrb_module = EnhancedAdaptiveMemoryRetrieval(
    hidden_size=768,
    k=5,
    memory_dim=768
)

grammar_module = EnhancedGrammarAwareModule(
    hidden_size=768,
    num_attention_heads=12,
    grammar_features_dim=64
)

print("✓ 核心模块初始化完成")
print(f"  - EWC模块: lambda={ewc_module.ewc_core.lambda_ewc}")
print(f"  - MRB模块: k={mrb_module.k}")
print(f"  - Grammar模块: dim={grammar_module.grammar_core.grammar_features_dim}")

## 4. 创建和组合策略

使用策略模式将各个模块组合成一个统一的持续学习流程。

In [ ]:
# 创建基础模型（不启用任何模块，我们将通过策略注入）
model = BERTCLAMModel(
    model_name='bert-base-uncased',
    num_labels=2,
    lora_r=8,
    lora_alpha=16,
    device=device,
    lora_enabled=True,
    enable_ewc=False,  # 通过策略启用
    enable_amr=False,  # 通过策略启用
    enable_grammar=False  # 通过策略启用
).to(device)

# 创建策略列表
strategies = [
    GrammarStrategy(grammar_module.to(device)),
    MRBStrategy(mrb_module.to(device), fusion_weight=0.2),
    EWCStrategy(ewc_module, model)
]

# 将策略注入模型
model.strategies = strategies

print("✓ 策略创建并注入完成")
print(f"  - 策略数量: {len(model.strategies)}")
for i, strategy in enumerate(model.strategies):
    print(f"  - 策略{i+1}: {strategy.name}")

## 5. 注入策略并训练

现在我们可以使用注入了策略的模型进行训练。这里我们进行一个极小规模的训练演示。

In [ ]:
# 设置优化器
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# 训练函数
def train_task(model, dataloader, task_id, num_steps=5):
    """训练单个任务"""
    model.train()
    model.register_task(task_id)
    
    total_loss = 0
    step = 0
    
    for batch in dataloader:
        if step >= num_steps:
            break
        
        # 将数据移到设备
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # 前向传播
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            task_id=task_id
        )
        
        loss = outputs['loss']
        
        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        step += 1
        
        print(f"  步骤 {step}/{num_steps}, 损失: {loss.item():.4f}")
    
    avg_loss = total_loss / step
    print(f"✓ 任务{task_id}训练完成，平均损失: {avg_loss:.4f}")
    
    # 保存任务检查点（用于EWC）
    model.save_task_checkpoint(task_id, dataloader)
    
    return avg_loss

# 训练任务0
print("\n=== 训练任务0 ===")
loss_task0 = train_task(model, task0_dataloader, task_id=0, num_steps=3)

# 训练任务1
print("\n=== 训练任务1 ===")
loss_task1 = train_task(model, task1_dataloader, task_id=1, num_steps=3)

print("\n✓ 所有任务训练完成！")

## 6. 评估和总结

让我们验证模型可以正常推理。

In [ ]:
# 评估函数
def evaluate(model, dataloader, task_id):
    """评估模型"""
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                task_id=task_id
            )
            
            predictions = torch.argmax(outputs['logits'], dim=-1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    
    accuracy = correct / total
    return accuracy

# 评估两个任务
print("\n=== 评估结果 ===")
acc_task0 = evaluate(model, task0_dataloader, task_id=0)
acc_task1 = evaluate(model, task1_dataloader, task_id=1)

print(f"任务0准确率: {acc_task0:.2%}")
print(f"任务1准确率: {acc_task1:.2%}")
print(f"平均准确率: {(acc_task0 + acc_task1) / 2:.2%}")

## 总结

在本教程中，您学习了：

1. ✓ 如何初始化BERT-CLAM框架的核心组件
2. ✓ 如何创建和组合持续学习策略
3. ✓ 如何将策略注入模型并进行训练
4. ✓ 如何在多个任务上进行持续学习

### 下一步

- 查看 `README.md` 了解更高级的用法
- 探索 `configs/` 目录中的配置文件示例
- 尝试在真实数据集上使用框架
- 自定义您自己的持续学习策略

### 相关资源

- [项目文档](../README.md)
- [配置文件示例](../configs/)
- [单元测试](../tests/)

祝您使用愉快！🚀